[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/RollingHedge/RollingHedge.ipynb)

# RollingHedge

Rolling hedge with collar strategy (cap + floor) and monthly cost comparison.

**Output:** `deriv_rolling_hedge.pdf`


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
# --- Generate realistic power price data ---
np.random.seed(42)

days = 365
t = np.arange(days)

# Mean-reverting process with seasonal component
base = 85 + 10 * np.sin(2 * np.pi * t / 365)  # seasonal
noise = np.zeros(days)
noise[0] = 0
for i in range(1, days):
    noise[i] = 0.92 * noise[i-1] + np.random.normal(0, 4)

# Add occasional spikes
spikes = np.random.choice([0, 1], size=days, p=[0.95, 0.05])
spike_mag = spikes * np.random.exponential(20, size=days)

power_price = base + noise + spike_mag
power_price = np.maximum(power_price, 30)  # floor at 30

# Collar parameters
cap = 100
floor_level = 70

# Hedged price: clipped to [floor, cap]
hedged_price = np.clip(power_price, floor_level, cap)


In [ ]:
# --- Two-panel chart: daily prices + monthly bars ---

dates = pd.date_range('2024-01-01', periods=days, freq='D')
df = pd.DataFrame({
    'unhedged': power_price,
    'hedged': hedged_price,
}, index=dates)

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# (A) Daily prices
ax = axes[0]
ax.plot(df.index, df['unhedged'], color=COLORS['blue'], lw=1.2, alpha=0.8, label='Unhedged Price')
ax.plot(df.index, df['hedged'], color=COLORS['green'], lw=1.5, label='Hedged Price (Collar)')
ax.axhline(cap, color=COLORS['red'], ls='--', lw=1, label=f'Cap = {cap}')
ax.axhline(floor_level, color=COLORS['orange'], ls='--', lw=1, label=f'Floor = {floor_level}')
ax.fill_between(df.index, floor_level, cap, alpha=0.08, color=COLORS['green'], label='Collar Band')
ax.set_title('(A) Daily Power Price: Unhedged vs Collar-Hedged', fontsize=13, fontweight='bold')
ax.set_ylabel('Price (EUR/MWh)')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=5)

# (B) Monthly average cost
monthly = df.resample('M').mean()
months = np.arange(len(monthly))
width = 0.35

ax2 = axes[1]
ax2.bar(months - width/2, monthly['unhedged'], width, color=COLORS['blue'], alpha=0.8, label='Unhedged Avg')
ax2.bar(months + width/2, monthly['hedged'], width, color=COLORS['green'], alpha=0.8, label='Hedged Avg')
ax2.set_xticks(months)
ax2.set_xticklabels([d.strftime('%b') for d in monthly.index], rotation=45)
ax2.set_title('(B) Monthly Average Cost Comparison', fontsize=13, fontweight='bold')
ax2.set_ylabel('Avg Price (EUR/MWh)')
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=2)

# Savings
total_unhedged = df['unhedged'].sum()
total_hedged = df['hedged'].sum()
savings_pct = (total_unhedged - total_hedged) / total_unhedged * 100
print(f"Total unhedged cost index: {total_unhedged:,.0f}")
print(f"Total hedged cost index:   {total_hedged:,.0f}")
print(f"Savings from collar hedge: {savings_pct:.1f}%")

fig.suptitle('Rolling Collar Hedge Strategy', fontsize=15, fontweight='bold', y=1.02)
fig.tight_layout()
save_fig(fig, 'deriv_rolling_hedge.pdf')
plt.show()
